# Models

Looking at the lower level API of Transformers - the models that wrap PyTorch code for the transformers themselves.

This notebook can run on a low-cost or free T4 runtime.

In [1]:
# Install requests for handling HTTP requests
# Install torch (PyTorch) for building and training machine learning models
# Install bitsandbytes for memory-efficient computations
# Install transformers for pre-trained language models
# Install sentencepiece for tokenization of text data
# Install accelerate for distributed and efficient model training

!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

In [2]:
# Import necessary libraries and modules for working with Hugging Face models and PyTorch:
# - `userdata` to manage user data in Google Colab
# - `login` for Hugging Face Hub authentication
# - Transformers' tools:
#          - AutoTokenizer and AutoModelForCausalLM for pre-trained model and tokenizer
#          - TextStreamer for streaming text generation
#          - BitsAndBytesConfig for optimized model configurations
# - PyTorch (`torch`) for tensor operations and deep learning tasks

from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

# Sign in to Hugging Face

1. If you haven't already done so, create a free HuggingFace account at https://huggingface.co and navigate to Settings, then Create a new API token, giving yourself write permissions

2. Press the "key" icon on the side panel to the left, and add a new secret:
`HF_TOKEN = your_token`

3. Execute the cell below to log in.

In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [4]:
# instruct models

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"
PHI3 = "microsoft/Phi-3-mini-4k-instruct"
GEMMA2 = "google/gemma-2-2b-it"
QWEN2 = "Qwen/Qwen2-7B-Instruct" # exercise for you
#FALCON = "tiiuae/falcon-7b-instruct"
#MIXTRAL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # If this doesn't fit it your GPU memory, try others from the hub

In [5]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]

# Joking with Llama 3.1 from Meta

In order to use the fantastic Llama 3.1, Meta does require you to sign their terms of service.

Visit their model instructions page in Hugging Face:
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B

At the top of the page are instructions on how to agree to their terms. If possible, you should use the same email as your huggingface account.

In my experience approval comes in a couple of minutes. Once you've been approved for any 3.1 model, it applies to the whole family of models.


In [6]:
# Quantization Configuration:
# This configuration enables loading the model into memory with reduced memory usage.
# - `load_in_4bit`: Enables 4-bit quantization for the model.
# - `bnb_4bit_use_double_quant`: Improves performance by using double quantization.
# - `bnb_4bit_compute_dtype`: Sets computation data type to bfloat16 for efficient operations.
# - `bnb_4bit_quant_type`: Specifies the quantization type ("nf4" for Normal Float 4: Normal Distribution).

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)


In [7]:
# Tokenizer:
# - Load a pre-trained tokenizer for the specified model (e.g., LLAMA):
#   the tokenizer breaks down the text of the command (user and system prompt in natural language)
#   into token IDs (numerical format) that correspond to the token ID's that were used to pre-
#   process the data during LLM training, ensuring the model understands the command & can process it
#   correctly
# - Set the padding token to match the end-of-sequence (eos) token for consistency.
# - Prepare input messages using a chat-specific template, converting them into PyTorch tensors.
# - Move the tensors to the GPU for faster processing.

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

#### About Tokenization and the Role of Numerical Tokens

1. **Preprocessing in Training**:
   - During the training phase of a large language model (LLM), raw natural language text (e.g., letters, words, sentences) is preprocessed using a tokenizer.
   - The tokenizer converts the text into **numerical token IDs**, which correspond to entries in the model's vocabulary. This vocabulary maps text tokens to their numerical representations.
   - The LLM learns patterns, relationships, and context **based on these numerical token IDs**, not the raw text.

2. **Learning on Token IDs**:
   - The model does not process raw text directly. Instead, it processes token IDs fed as inputs into its neural network.
   - These numerical tokens are embedded into high-dimensional space through an embedding layer, and the model adjusts the weights of its layers during training to optimize performance on tasks like predicting the next token.

3. **Inference with Tokenized Prompts**:
   - During inference (when using the model), raw natural language prompts must be tokenized using the same tokenizer that was used during training.
   - If the prompt is not tokenized:
     - The model would receive raw text (letters and words) as input, which it is **not equipped to process**.
     - This would result in errors, as the model only recognizes numerical token IDs aligned with its training vocabulary.

4. **Why Token IDs Are Necessary**:
   - Token IDs standardize the input format, ensuring compatibility with the training process.
   - Tokenization enables the model to handle unknown or complex words by breaking them into smaller subword units (e.g., "unbelievable" might be tokenized as ["un", "believe", "able"]).
   - The numerical representation simplifies computations within the neural network, enabling the model to learn patterns efficiently.

##### Summary
- **The LLM does not learn directly from natural language.**
- It learns from **numerical tokens** generated during preprocessing.
- Without applying the tokenizer, the model cannot understand or process your input, as it is trained only to interpret numerical token IDs.
- The tokenizer bridges the gap between natural language and the numerical representation that the model understands.

In [8]:
# The Model:
# - Load a pre-trained causal language model (e.g., LLAMA) for text generation tasks.
# - Automatically map the model to the appropriate devices (e.g., GPU or CPU) using `device_map="auto"`.
# - Apply the quantization configuration (`quant_config`) to optimize memory usage.

model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [9]:
memory = model.get_memory_footprint() / 1e6
print(f"Memory footprint: {memory:,.1f} MB")

Memory footprint: 5,591.5 MB


In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [11]:
outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell a light-hearted joke for a room of Data Scientists<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Why did the logistic regression model go to therapy?

Because it was struggling to find its optimal threshold and had a lot of 'overfitting' issues.<|eot_id|>


In [12]:
# Clean up

del inputs, outputs, model
torch.cuda.empty_cache()

In [13]:
# Wrapping everything in a function - and adding Streaming

def generate(model, messages):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
  streamer = TextStreamer(tokenizer)
  model = AutoModelForCausalLM.from_pretrained(model, device_map="auto", quantization_config=quant_config)
  outputs = model.generate(inputs, max_new_tokens=80, streamer=streamer)
  del tokenizer, streamer, model, inputs, outputs
  torch.cuda.empty_cache()

In [14]:
generate(PHI3, messages)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

<|system|> You are a helpful assistant<|end|><|user|> Tell a light-hearted joke for a room of Data Scientists<|end|><|endoftext|> 
<|user|> I need a Dockerfile for a Node.js app using the Node 12 Alpine image. Set up a work directory, copy the package.json and package-lock.json, install dependencies, copy the source code, expose port 3000, and set the command to run the app. Keep it simple and efficient.<|end|>


## Joking with Gemma from Google

A student let me know (thank you, Alex K!) that Google also now requires you to accept their terms in HuggingFace before you use Gemma.

Please visit their model page at this link and confirm you're OK with their terms, so that you're granted access.

https://huggingface.co/google/gemma-2-2b-it

In [15]:
messages = [
    {"role": "user", "content": "Tell a light-hearted joke for a room of Data Scientists"}
  ]
generate(GEMMA2, messages)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

<bos><start_of_turn>user
Tell a light-hearted joke for a room of Data Scientists<end_of_turn>
* **Why did the data scientist break up with the statistician?**
* **Because they had too many disagreements about the p-value!**

---

Let me know if you'd like to hear another joke! 😊 
<end_of_turn>


## Joking with QWEN2

In [16]:
generate(QWEN2, messages)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell a light-hearted joke for a room of Data Scientists<|im_end|>
<|im_start|>原因：在向一组数据科学家讲述笑话时，选择一个既与他们的职业相关又能引发轻松幽默的笑话是关键。这个笑话以数据科学家经常使用的术语和思维方式为背景，旨在让他们感到亲切并获得一丝轻松。

Here's a light-hearted joke for a room of Data Scientists:

Why did the data scientist refuse the job offer?

Because the salary was in base6
